In [16]:
import os
import numpy as np
import pandas as pd
import cv2
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils

# Define directories
image_directory = '../data/fish-behaviour/annotations/images/'  # Original Images
depth_map_directory = '../data/fish-behaviour/depth-maps/'  # Depth Map Images
mask_directory = '../data/fish-behaviour/masks/'    # Directory to save masks
json_file = '../data/fish-behaviour/annotations/annotations/instances_default.json'  # JSON file

# Create mask directory if it doesn't exist
os.makedirs(mask_directory, exist_ok=True)

# Load the COCO annotations
coco = COCO(json_file)

In [17]:
# Change Model Path
MODEL = "hypersim"
ENCODER = "vitl"

In [18]:
# Define directories
image_directory = '../data/fish-behaviour/annotations/images/'  # Original Images
depth_map_directory = '../data/fish-behaviour/depth-maps/'  # Depth Map Images from V1
mask_directory = '../data/fish-behaviour/masks/'    # Directory to save masks
csv_file = "../data/fish-behaviour/fish-behaviour.csv"  # CSV file
json_file = '../data/fish-behaviour/annotations/annotations/instances_default.json'  # JSON file

# Create mask directory if it doesn't exist
os.makedirs(mask_directory, exist_ok=True)

In [19]:
# Load the COCO annotations
coco = COCO(json_file)

# Load the CSV file
data = pd.read_csv(csv_file)

# Display the first few rows of the DataFrame
print("Data from CSV:")
print(data.head())

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Data from CSV:
Empty DataFrame
Columns: [test]
Index: []


In [20]:
# Create a mapping from image base name (without extension) to frame number
data['image_base_name'] = data['image_name'].apply(lambda x: os.path.splitext(x)[0])
image_frame_mapping = data.set_index('image_base_name')['frame'].to_dict()


KeyError: 'image_name'

In [21]:
# For debugging, print out the keys of image_frame_mapping
print("Keys in image_frame_mapping:")
print(list(image_frame_mapping.keys())[:5])

Keys in image_frame_mapping:


NameError: name 'image_frame_mapping' is not defined

In [12]:
def load_depth_map(img_name):
    # Construct the depth map filename
    depth_map_filename = f"{img_name}_pred04.png"
    depth_map_path = os.path.join(depth_map_directory, depth_map_filename)
    
    # Check if the depth map exists
    if not os.path.exists(depth_map_path):
        print(f"Depth map for image {img_name} not found at {depth_map_path}.")
        return None
    
    # Load the depth map image
    depth_map = cv2.imread(depth_map_path, cv2.IMREAD_GRAYSCALE)
    if depth_map is None:
        print(f"Failed to load depth map for image {img_name}.")
        return None
    
    return depth_map

In [13]:
def create_mask_from_annotation(ann, img_height, img_width):
    segmentation = ann['segmentation']
    
    if isinstance(segmentation, list):  # Polygon format
        mask = np.zeros((img_height, img_width), dtype=np.uint8)
        # Convert polygons to a mask
        for poly in segmentation:
            poly = np.array(poly).reshape((-1, 2))  # Reshape to (N, 2)
            cv2.fillPoly(mask, [np.int32(poly)], 1)  # Fill the polygon with 1's

    elif isinstance(segmentation, dict) and 'counts' in segmentation:  # RLE format
        rle = segmentation
        if isinstance(rle['counts'], list):  # Uncompressed RLE
            rle = maskUtils.frPyObjects([rle], img_height, img_width)
        mask = maskUtils.decode(rle)  # Decode RLE to binary mask

    else:
        mask = None  # Segmentation format not recognized

    return mask

In [14]:
# List to store results
results = []

# Get all image IDs
image_ids = coco.getImgIds()
print(f"Total number of images: {len(image_ids)}")

# Process each image
for img_id in image_ids:
    # Load image information
    img_info = coco.loadImgs(img_id)[0]
    img_name_with_ext = img_info['file_name']  # e.g., 'frame_000001.png'
    img_name = os.path.splitext(img_name_with_ext)[0]  # e.g., 'frame_000001'
    img_height = img_info['height']
    img_width = img_info['width']
    
    print(f"Processing image ID {img_id}, name {img_name_with_ext}")

    # Load depth map
    depth_map = load_depth_map(img_name)
    if depth_map is None:
        print(f"Skipping image {img_name_with_ext} due to missing depth map.")
        continue  # Skip if depth map is not available
    
    # Get the frame number from the image_frame_mapping
    frame = image_frame_mapping.get(img_name, None)
    if frame is None:
        print(f"Frame number not found for image {img_name_with_ext}.")
    else:
        print(f"Frame number for image {img_name_with_ext}: {frame}")
    
    # Get annotation IDs for the image
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    
    print(f"Number of annotations for image {img_name_with_ext}: {len(anns)}")
    
    # Process each annotation
    for ann in anns:
        annotation_id = ann['id']
        category_id = ann['category_id']
        print(f"Processing annotation ID {annotation_id}, category ID {category_id}")
        
        # Create mask for the annotation
        mask = create_mask_from_annotation(ann, img_height, img_width)
        if mask is None:
            print(f"Failed to create mask for annotation {annotation_id}")
            continue  # Skip if mask couldn't be created
        
        # Apply the mask to the depth map to extract depth values
        depth_values = depth_map[mask == 1]
        mean_depth = depth_values.mean() if depth_values.size > 0 else 0
        normalized_depth = mean_depth / 255  # Assuming 8-bit depth map
        
        # Convert normalized depth to metric depth
        zmin = 0.4  # Minimum depth in meters
        zmax = 20.0  # Maximum depth in meters
        value_metric = (normalized_depth * (zmax - zmin)) + zmin
        
        print(f"Annotation ID {annotation_id}: mean depth {mean_depth}, normalized depth {normalized_depth}, value_metric {value_metric}")
        
        # Append the result to the list
        results.append({
            'model': "indoor",
            'image_name': img_name_with_ext,
            'image_id': img_id,
            'annotation_id': annotation_id,
            'frame': frame,
            'category_id': category_id,
            'normalized_depth': normalized_depth,
            'z-min': zmin,
            'z-max': zmax,
            'value_metric': value_metric
        })

NameError: name 'image_frame_mapping' is not defined

In [15]:
# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Display the DataFrame
print("Results DataFrame:")
print(results_df.head()

""


In [142]:
# Save the results to a CSV file
results_df.to_csv(f'../data/fish-behaviour/fish-behaviour.csv', index=False)